## Evaluation from generated doc

In [1]:
import pandas as pd
import numpy as np
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
from itertools import combinations
from collections import Counter

In [73]:
# ✅ Load topic results
df = pd.read_csv("gpu_results5.csv")

In [74]:
# Ensure all entries in 'keywords' are lists, replacing NaN (floats) with empty lists
df['keywords'] = df['keywords'].apply(lambda x: x if isinstance(x, str) else [])

In [75]:
df

,name,percentage,keywords,document_index,percentence,percenture,key keywords
0,Historical Debates and Controversies,40.0,"['Turks', 'Armenians', 'Holocaust', 'extermina...",15001,NaN,NaN,NaN
1,Artificial Intelligence and its Misuse,20.0,"['AI', 'stupidity', 'amok', 'compose', 'pointl...",15001,NaN,NaN,NaN
2,Intercultural Communication and Stereotypes,15.0,"['insults', 'Typical Mutlu', 'PvdL', 'Cosar']",15001,NaN,NaN,NaN
3,Internet Discourse and Netiquette,10.0,"['article', 'followup', 'alleviate puzzlement'...",15001,NaN,NaN,NaN
4,Political Opinion and Bias,10.0,"['Turks did what they are accused of', 'Armeni...",15001,NaN,NaN,NaN
...,...,...,...,...,...,...,...
6017,Celebrity and Fame,35.0,"['Jesse', 'famous', 'intern']",16309,NaN,NaN,NaN
6018,Workplace Relationships,25.0,"['work', 'intern', 'Landau']",16309,NaN,NaN,NaN
6019,Spite and Competition,15.0,"['like it out of spite', 'Just kidding']",16309,NaN,NaN,NaN
6020,Quotation Analysis,10.0,"['skepticism', 'intellect', 'Gordon Banks']",16309,NaN,NaN,NaN


In [76]:
# Flatten the keyword list and count occurrences
all_keywords = [word for sublist in df['keywords'] for word in sublist]
word_counts = Counter(all_keywords)

In [77]:
# Compute co-occurrence counts
co_occurrence_counts = Counter()
for keyword_list in df['keywords']:
    for word1, word2 in combinations(keyword_list, 2):
        co_occurrence_counts[frozenset([word1, word2])] += 1

In [78]:
# Remove invalid entries (pairs with fewer than 2 elements)
invalid_entries = [pair for pair in co_occurrence_counts.keys() if len(pair) != 2]
for pair in invalid_entries:
    del co_occurrence_counts[pair]

In [79]:
# Compute NPMI values
npmi_scores = {}
total_pairs = sum(co_occurrence_counts.values())

In [80]:
for word_pair, co_occ_count in co_occurrence_counts.items():
    word1, word2 = list(word_pair)
    p_x = word_counts[word1] / len(all_keywords)
    p_y = word_counts[word2] / len(all_keywords)
    p_xy = co_occ_count / total_pairs

    if p_xy > 0:
        pmi = np.log(p_xy / (p_x * p_y))
        npmi = pmi / -np.log(p_xy)
        npmi_scores[word_pair] = npmi

In [81]:
# Compute Topic Diversity (TD) based on topic names
df_grouped = df.groupby("document_index")["name"].apply(list)
all_topic_names = [name for sublist in df_grouped for name in sublist]
unique_topic_names = set(all_topic_names)
total_topic_names = len(all_topic_names)
topic_diversity_names = len(unique_topic_names) / total_topic_names

In [82]:
# Compute average NPMI
npmi_avg = np.mean(list(npmi_scores.values())) if npmi_scores else 0

In [83]:
npmi_avg

0.08467779335128871

In [84]:
topic_diversity_names

0.8771172367984058

In [69]:
# Save results
results = {
    "NPMI Score": npmi_avg,
    "Topic Diversity": topic_diversity_names
}
pd.DataFrame([results]).to_csv("bbc_mistral_evaluation.csv", index=False)